In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
teams = pd.read_csv("./Teams.csv")
teamspel = pd.read_csv("./TeamSpellings.csv")
kp = pd.read_csv("./kenpom.csv")

In [3]:
teamspel.head(50)

,name_spelling,team_id
0,a&m-corpus chris,1394
1,abilene chr,1101
2,abilene christian,1101
3,abilene-christian,1101
4,air force,1102
5,air-force,1102
6,akron,1103
7,alabama,1104
8,alabama a&m,1105
9,alabama st,1106


In [4]:
kp2017 = kp[kp['Year']==2017]
kp2017 = kp2017.reset_index(drop=True)

In [5]:
kp2017['Team'] = kp2017['Team'].str.lower()
kp2017['Team'] = kp2017['Team'].str.strip()

In [6]:
kp2017.head()

,Year,Rank,Team,Conference,Wins,Losses,Seed,Pyth,AdjustO,AdjustO Rank,...,Luck,Luck Rank,SOS Pyth,SOS Pyth Rank,SOS OppO,SOS OppO Rank,SOS OppD,SOS OppD Rank,NCSOS Pyth,NCSOS Pyth Rank
0,2017,1,gonzaga,WCC,32,1,1.0,32.88,120.6,10,...,0.016,148,0.52,130,104.8,139,104.3,129,1.02,132
1,2017,2,villanova,BE,31,3,1.0,30.65,122.6,2,...,0.026,116,9.83,27,110.1,28,100.3,34,4.14,54
2,2017,3,north carolina,ACC,27,7,1.0,27.86,122.0,4,...,0.006,174,11.95,10,111.5,6,99.6,19,3.92,57
3,2017,4,kentucky,SEC,29,5,2.0,26.93,119.0,14,...,0.008,168,9.42,33,109.3,39,99.9,24,3.69,62
4,2017,5,west virginia,B12,26,8,4.0,26.83,116.5,27,...,-0.043,288,9.77,28,109.3,38,99.6,16,-7.77,341


In [8]:
kp2017id = kp2017.merge(teamspel, how='left', left_on='Team', right_on='name_spelling')

In [10]:
ssub = pd.read_csv("./SampleSubmission.csv")

In [11]:
ssub['year'] = ssub.Id.str.split('_').str.get(0)
ssub['team1'] = ssub.Id.str.split('_').str.get(1)
ssub['team2'] = ssub.Id.str.split('_').str.get(2)
ssub.drop('Pred', axis=1, inplace=True)
ssub[['year', 'team1', 'team2']] = ssub[['year', 'team1', 'team2']].apply(pd.to_numeric)
ss17 = ssub

In [12]:
t1_2017_end = pd.DataFrame()
t1_2017_end[['t1_id', 't1_ncsos_pyth', 't1_sos_oppd', 't1_sos_oppo', 't1_sospyth', 't1_luck',
                          't1_adjustt','t1_adjusto', 't1_adjustd', 't1_pyth']] = kp2017id[['team_id', 'NCSOS Pyth',
                          'SOS OppD', 'SOS OppO', 'SOS Pyth', 'Luck',
                          'AdjustT','AdjustO', 'AdjustD', 'Pyth']]

In [13]:
t2_2017_end = pd.DataFrame()
t2_2017_end[['t2_id', 't2_ncsos_pyth', 't2_sos_oppd', 't2_sos_oppo', 't2_sospyth', 't2_luck',
                          't2_adjustt','t2_adjusto', 't2_adjustd', 't2_pyth']] = kp2017id[['team_id', 'NCSOS Pyth',
                          'SOS OppD', 'SOS OppO', 'SOS Pyth', 'Luck',
                          'AdjustT','AdjustO', 'AdjustD', 'Pyth']]

In [14]:
t1_2017_end.tail()

,t1_id,t1_ncsos_pyth,t1_sos_oppd,t1_sos_oppo,t1_sospyth,t1_luck,t1_adjustt,t1_adjusto,t1_adjustd,t1_pyth
346,1255,-0.31,107.0,101.6,-5.39,0.017,70.1,92.3,117.2,-24.98
347,1115,3.11,109.4,100.5,-8.99,-0.010,67.2,85.7,110.8,-25.17
348,1299,0.47,107.9,98.8,-9.05,-0.086,62.4,95.5,123.1,-27.61
349,1342,2.75,106.9,103.0,-3.89,0.018,63.8,89.0,118.0,-28.99
350,1105,1.37,108.7,98.8,-9.80,-0.075,63.8,91.6,121.9,-30.25


In [16]:
ss17 = ss17.merge(t1_2017_end, how='left', left_on='team1', right_on='t1_id')

In [18]:
ss17 = ss17.merge(t2_2017_end, how='left', left_on='team2', right_on='t2_id')

In [19]:
ss17.count()

Id               2278
year             2278
team1            2278
team2            2278
t1_id            2278
t1_ncsos_pyth    2278
t1_sos_oppd      2278
t1_sos_oppo      2278
t1_sospyth       2278
t1_luck          2278
t1_adjustt       2278
t1_adjusto       2278
t1_adjustd       2278
t1_pyth          2278
t2_id            2278
t2_ncsos_pyth    2278
t2_sos_oppd      2278
t2_sos_oppo      2278
t2_sospyth       2278
t2_luck          2278
t2_adjustt       2278
t2_adjusto       2278
t2_adjustd       2278
t2_pyth          2278
dtype: int64

In [20]:
ss17.tail()

,Id,year,team1,team2,t1_id,t1_ncsos_pyth,t1_sos_oppd,t1_sos_oppo,t1_sospyth,t1_luck,...,t2_id,t2_ncsos_pyth,t2_sos_oppd,t2_sos_oppo,t2_sospyth,t2_luck,t2_adjustt,t2_adjusto,t2_adjustd,t2_pyth
2273,2017_1455_1458,2017,1455,1458,1455,-0.08,103.7,104.4,0.67,-0.014,...,1458,-1.08,100.9,109.8,8.96,-0.00,63.0,113.5,91.3,22.20
2274,2017_1455_1462,2017,1455,1462,1455,-0.08,103.7,104.4,0.67,-0.014,...,1462,5.55,99.9,110.7,10.86,0.03,66.9,114.7,99.6,15.16
2275,2017_1457_1458,2017,1457,1458,1457,2.34,107.9,100.4,-7.50,0.029,...,1458,-1.08,100.9,109.8,8.96,-0.00,63.0,113.5,91.3,22.20
2276,2017_1457_1462,2017,1457,1462,1457,2.34,107.9,100.4,-7.50,0.029,...,1462,5.55,99.9,110.7,10.86,0.03,66.9,114.7,99.6,15.16
2277,2017_1458_1462,2017,1458,1462,1458,-1.08,100.9,109.8,8.96,-0.000,...,1462,5.55,99.9,110.7,10.86,0.03,66.9,114.7,99.6,15.16


In [21]:
rsdr = pd.read_csv("./RegularSeasonDetailedResults.csv")

In [22]:
rs2017 = rsdr[rsdr['Season']==2017]
rs2017_d1 = pd.DataFrame()
rs2017_d2 = pd.DataFrame()
rs2017_d1[['team1', 'team2', 'Daynum']] = rs2017[['Wteam', 'Lteam', 'Daynum']].copy()
rs2017_d2[['team1', 'team2', 'Daynum']] = rs2017[['Lteam', 'Wteam', 'Daynum']].copy()
rs2017_d1['pred'] = 1
rs2017_d2['pred'] = 0
rs2017_train = pd.concat([rs2017_d1, rs2017_d2]).reset_index(drop=True)

In [31]:
kp_feat_17 = rs2017_train.merge(t1_2017_end, how='left', left_on='team1', right_on='t1_id')

In [33]:
kp_feat_17 = kp_feat_17.merge(t2_2017_end, how='left', left_on='team2', right_on='t2_id')

In [35]:
kp_feat_17.tail()

,team1,team2,Daynum,pred,t1_id,t1_ncsos_pyth,t1_sos_oppd,t1_sos_oppo,t1_sospyth,t1_luck,...,t2_id,t2_ncsos_pyth,t2_sos_oppd,t2_sos_oppo,t2_sospyth,t2_luck,t2_adjustt,t2_adjusto,t2_adjustd,t2_pyth
10785,1458,1276,132,0,1458,-1.08,100.9,109.8,8.96,-0.000,...,1276,-0.35,99.6,109.2,9.63,-0.031,62.5,121.7,99.1,22.57
10786,1463,1343,132,0,1463,-0.96,105.5,103.8,-1.72,0.055,...,1343,2.35,104.9,103.4,-1.52,0.025,62.9,109.4,96.9,12.45
10787,1433,1348,132,0,1433,1.16,102.0,105.5,3.47,0.036,...,1348,1.92,102.8,106.3,3.53,-0.024,65.9,110.6,95.1,15.49
10788,1153,1374,132,0,1153,-0.26,103.1,106.5,3.37,0.029,...,1374,-0.96,101.9,104.9,3.04,0.020,63.3,120.0,94.9,25.15
10789,1402,1407,132,0,1402,-8.07,106.4,102.7,-3.71,0.019,...,1407,-6.63,105.3,102.1,-3.23,-0.037,67.7,109.8,107.5,2.33


In [26]:
rs2017_train.count()

team1     10790
team2     10790
Daynum    10790
pred      10790
dtype: int64

In [36]:
target17 = kp_feat_17['pred'].values

In [37]:
features_array17 = kp_feat_17[['t1_ncsos_pyth', 't1_sos_oppd', 't1_sos_oppo', 't1_sospyth', 't1_luck', 't1_adjustt',
                               't1_adjusto', 't1_adjustd', 't1_pyth', 't2_ncsos_pyth', 't2_sos_oppd', 't2_sos_oppo',
                               't2_sospyth', 't2_luck', 't2_adjustt', 't2_adjusto', 't2_adjustd', 't2_pyth']].values

In [38]:
test_17 = ss17[['t1_ncsos_pyth', 't1_sos_oppd', 't1_sos_oppo', 't1_sospyth', 't1_luck',
                't1_adjustt','t1_adjusto', 't1_adjustd', 't1_pyth', 't2_ncsos_pyth',
                't2_sos_oppd', 't2_sos_oppo', 't2_sospyth', 't2_luck',
                't2_adjustt','t2_adjusto', 't2_adjustd', 't2_pyth']].values

In [39]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, subsample=.8, max_features=.5)

In [40]:
gb.fit(features_array17, target17)
t1_pred_prob_2017 = gb.predict_proba(test_17)

In [41]:
preds17 = t1_pred_prob_2017[:,1]
ss17['pred'] = preds17

In [42]:
ss17 = ss17[['Id', 'pred']]
ss17 = ss17.sort_values('Id').reset_index(drop=True)
ss17.tail()

,Id,pred
2273,2017_1455_1458,0.530584
2274,2017_1455_1462,0.616735
2275,2017_1457_1458,0.079680
2276,2017_1457_1462,0.101307
2277,2017_1458_1462,0.678168


In [43]:
ss17.to_csv('kp_gb_2017.csv', index=False)